# Assigment 03

Now that we already have the data from previous assgiment, let import the files and start to work with them.

In [1]:
#Import the librarys
import pandas as pd

In [2]:
data = pd.read_pickle('./data/raw/10-K_Data.pik')

Lets Just take 10 files to process in this assigment

In [16]:
data = pd.DataFrame(data.sample(10), copy=True)
data.reset_index(inplace=True)

## Removing the html tags

The files we have imported have html tags and it is very dificult to read, lets tale a look

In [3]:
sample = data['corpus'][0]
print(sample[300:700])

b'15\nDATE AS OF CHANGE:\t\t20170315\n\nFILER:\n\n\tCOMPANY DATA:\t\n\t\tCOMPANY CONFORMED NAME:\t\t\tTRAVELZOO INC\n\t\tCENTRAL INDEX KEY:\t\t\t0001133311\n\t\tSTANDARD INDUSTRIAL CLASSIFICATION:\tSERVICES-COMPUTER INTEGRATED SYSTEMS DESIGN [7373]\n\t\tIRS NUMBER:\t\t\t\t364415727\n\t\tSTATE OF INCORPORATION:\t\t\tDE\n\t\tFISCAL YEAR END:\t\t\t1231\n\n\tFILING VALUES:\n\t\tFORM TYPE:\t\t10-K\n\t\tSEC ACT:\t\t1934 Act\n\t\tSEC FILE NUMBER:\t000-50171\n\t\tFILM N'


In [4]:
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re

In [5]:
def text_process( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string and the output is a single string
    
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review,"html5lib").get_text() 
    
    # 2. Remove non-letters
    review_text = re.sub('\s+', ' ',review_text)    
    letters_only = re.sub("[^a-zA-Z.]", " ", review_text)

    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words )) 

In [7]:
# Check to make sure its working
sample_clean = data['corpus'].head(1).apply(text_process)
list(sample_clean)[0][300:700]

'l year end filing values form type k sec act act sec file number film number business address street ave stars street th floor city los angeles state ca zip business phone mail address street ave stars street th floor city los angeles state ca zip k aresmanagement k.htm k document table contents united statessecurities exchange commissionwashington d.c. form k annual report pursuant section securi'

In [19]:
# Get the number of documents based on the dataframe column size
num_documents = data['corpus'].count()
corpus = data['corpus']
clean_corpus = []

# Loop over each document; create an index i that goes from 0 to the 
#length of documents 
for i in range( 0, num_documents ):
    # Call our function for each one, and add the result to the 
    #list of clena documents
    clean_corpus.append( text_process( corpus[i] ) )
    
    # Printing out the the progress. 
    try:
        count += 1
        end = time.time()
    except:
        import sys 
        import time
        start = time.time()
        end = time.time()
        count = 1
        
        #Need to change this variavel to match with the range of the for loop
        num_documents = data['corpus'].count() 
        
    sys.stdout.write("\rProgress: {:2.1f}".format(100 * count/float(num_documents)) \
                     + "% ... Documents processed: " + str(count) \
                     + " time running: " + str(int((end-start)/60)) + " minutes.") 
    sys.stdout.flush()

Progress: 100.0% ... Documents processed: 10 time running: 5 minutes.

In [20]:
data['corpus'] = clean_corpus

Lets save the file already clened, so next time we can start from here.

In [21]:
data.to_pickle('./data/10k-clean_sample10.pik')

# Importing the new data file

In [1]:
import pandas as pd
data = pd.read_pickle('./data/10k-clean_sample10.pik')

In [2]:
# This Library Calculates the FOG index. https://github.com/shivam5992/textstat
from textstat.textstat import textstat
#NLTK 
import nltk
from nltk.corpus import cmudict

In [3]:
#This variable d will store the dictionary CMU
d = cmudict.dict()

#This funcition return the count of syllables from a word that exists in the dictionary
def nsyl(word):
    return int([len(
                list(y for y in x 
                     if y[-1].isdigit())
                ) 
            for x in d[word.lower()]
           ][0])

In [4]:
#Creating the variables to store the data
sentence_count = []
sentence_average = [] 
syllable_count = []
complex_words = []
word_count = []
fog_index = []

In [5]:
#This will process the text
for corpus in data['corpus']:
    
    #sentences count
    sentencecount = len(corpus.split('. '))
       
    #sentences averages
    wordcount = 0
    for word in corpus.split(' '):
        if word in d:
            wordcount += 1
        
    setenceaverage = round(wordcount/sentencecount,2)
    
    #syllables
    syllablecount = 0
    complexwords = 0
    for word in corpus.split(' '):
        if word in d:
            syllables = nsyl(word)
            
            #Adding to count syllables average
            syllablecount += syllables
            
            #Count only word with more than 2 syllables
            if syllables >= 3:
                complexwords += 1
            
                
    syllablecount = round(syllablecount/wordcount,2) 
        
    #FOG
    fogindex = round(textstat.gunning_fog(corpus),2)
      
    #Append values to a list
    sentence_count.append(sentencecount)
    sentence_average.append(setenceaverage)
    syllable_count.append(syllablecount)
    complex_words.append(complexwords)
    word_count.append(wordcount)
    fog_index.append(fogindex)    

Progress: 100.0% ... Documents processed: 10 time running: 1 minutes.

In [6]:
#Adding the data to the dataframe    
data['sentence count'] = sentence_count
data['sentence average'] = sentence_average
data['syllables average'] = syllable_count
data['complex words'] = complex_words
data['word count '] = word_count
data['FOG index'] = fog_index

In [7]:
data.head(10)

,level_0,index,company_name,form_type,cik,date,file,corpus,sentence count,sentence average,syllables average,complex words,word count,FOG index
0,60,142433,"ICON LEASING FUND TWELVE, LLC",10-K,1377848,2017-03-30,edgar/data/1377848/0001377848-17-000003.txt,.txt .hdr.sgml accession number conformed subm...,9721,27.33,1.75,55263,265693,14.72
1,37,107981,FS Investment Corp II,10-K,1525759,2017-03-15,edgar/data/1525759/0001193125-17-083729.txt,.txt .hdr.sgml accession number conformed subm...,5634,12.01,2.12,24547,67680,10.11
2,48,11986,ARES MANAGEMENT LP,10-K,1176948,2017-02-27,edgar/data/1176948/0001628280-17-001756.txt,.txt .hdr.sgml accession number conformed subm...,25146,39.93,1.70,184741,1004108,21.16
3,61,194217,Momentive Performance Materials Inc.,10-K,1405041,2017-03-10,edgar/data/1405041/0001405041-17-000006.txt,.txt .hdr.sgml accession number conformed subm...,12995,50.29,1.69,118039,653539,23.76
4,58,172083,"LegacyTexas Financial Group, Inc.",10-K,1487052,2017-02-09,edgar/data/1487052/0001487052-17-000019.txt,.txt .hdr.sgml accession number conformed subm...,15061,37.04,1.66,99435,557885,19.00
5,26,250025,"SILVER BULL RESOURCES, INC.",10-K,1031093,2017-01-19,edgar/data/1031093/0001079973-17-000033.txt,.txt .hdr.sgml accession number conformed subm...,6800,21.00,1.85,32309,142767,12.18
6,96,211260,ORIGINAL SIXTEEN TO ONE MINE INC /CA/,10-K,74925,2017-03-27,edgar/data/74925/0000074925-17-000001.txt,.txt .hdr.sgml accession number conformed subm...,399,9.20,2.19,1344,3672,18.43
7,91,147188,Inbit Corp,10-K,1657803,2017-03-27,edgar/data/1657803/0001493152-17-002853.txt,.txt .hdr.sgml accession number conformed subm...,1422,16.39,1.85,6174,23309,10.78
8,68,183226,"MIMEDX GROUP, INC.",10-K,1376339,2017-03-01,edgar/data/1376339/0001376339-17-000042.txt,.txt .hdr.sgml accession number conformed subm...,9840,26.38,1.71,52046,259557,14.36
9,90,182267,MGM Growth Properties LLC,10-K,1656936,2017-03-06,edgar/data/1656936/0001628280-17-002206.txt,.txt .hdr.sgml accession number conformed subm...,9045,32.40,1.71,57879,293060,16.31
